FIRST WIN OF THE WWE CHAMPIONSHIP

Lo scopo di questo Notebook è mettere in relazione il periodo in cui i wrestler della WWF/WWE vinsero il loro primo titolo major (WWE Championship) con l'età alla quale conseguirono tale traguardo.

Il motivo di questa analisi è quello di capire se, come succede per altri sport (vedere calcio o sport motoristici), con il succedersi delle generazioni consegue un ringiovanimento dei campioni di tale sport.

Importiamo dunque librerie utili al nostro obiettivo.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_rows', None)

Importiamo il dataset, mantenendo solo le entrate relative alla prima volta in cui un wrestler vinse il titolo più importante (WWE Championship).

In [ ]:
ds = pd.read_csv('../input/professional-wrestling-champions-wwewwf/data.csv')

ds = ds[:140]
ds = ds.drop([0,1,109])

for i in ds.index:
    if ds.reign[i]>1:
        ds = ds.drop(i)
        
ds

Trasformiamo le colonne 'date' e 'date_of_birth' nello stesso formato e calcoliamo l'età alla quale il wrestler vinse il titolo per la prima volta.

In [ ]:
ds['date_of_birth'] = pd.to_datetime(ds['date_of_birth'])
ds['date'] = pd.to_datetime(ds['date'])

x = ds['date']
y = ds.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)


Una volta istanziate le date dei match in x e l'età in y, rappresentiamo il grafico con i punti relativi alle varie entrate del dataset.

In [ ]:
plt.figure(figsize=(30, 12))
plt.scatter(x,y)

plt.xlabel('Title Win Date')
plt.ylabel('Age')

Importiamo ora le librerie necessarie per implementare il metodo Ridge Regression, facendo attenzione a normalizzare le date.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

x_0 = np.array(x)
x_0 = StandardScaler().fit_transform(x_0.reshape(-1, 1))



Implementiamo la regressione polinomiale mediante polinomi di grado diverso, al momento ponendo $\alpha=0$. 

In [ ]:
plt.figure(figsize=(25, 12))
plt.scatter(x,y)

plt.xlabel('Title Win Date')
plt.ylabel('Age')

for degree in [1, 2, 3, 5, 10]:
    model = make_pipeline(PolynomialFeatures(degree), Ridge(alpha = 0))
    model.fit(x_0,y)
    y_pred = model.predict(x_0)
    plt.plot(x, y_pred, label = 'degree = ' + str(degree))

plt.legend()

Come si può notare, i polinomi di grado più alto tentano di avvicinarsi di più ai dati di partenza del dataset, tuttavia essi propongono un andamento poco probabile in coerenza con eventuali nuovi dati.

Consideriamo, quindi, il polinomio di grado 10. Ora variamo $\alpha$ secondo le potenze di 10, mantenendo fissato il grado del polinomio.

In [ ]:
plt.figure(figsize=(25, 12))
plt.scatter(x,y)

plt.xlabel('Title Win Date')
plt.ylabel('Age')

colors = ["purple", "red", "pink", "orange", "black"]
i=0

for alpha in [0, 1, 10, 100, 1000]:
    model = make_pipeline(PolynomialFeatures(10), Ridge(alpha = alpha))
    model.fit(x_0,y)
    y_pred = model.predict(x_0)
    plt.plot(x, y_pred, color = colors[i], label = 'alpha = ' + str(alpha))
    i+=1
    
plt.legend()

All'aumentare di $\alpha$ il polinomio viene 'ammorbidito' soprattutto in corrispondenza dei periodi in cui si hanno maggiori dati.

Considerando il lasso di tempo 1975-2015 otteniamo un andamento costante con età comprese fra i 32/33 e i 35 anni, eppure leggermente decrescente.
Data, invece, la scarsità relativa ai periodi precedenti al 1975 e successivi al 2015, l'effetto di $\alpha$ è limitato.

La conclusione di questa analisi è che la gavetta dei wrestler è ben più lunga rispetto ad altri sport dove spesso si attesta ben sotto i 30 anni.
Inoltre l'andamento non è "abbastanza" decrescente per poter individuare un "ringiovanimento" dei campioni di wrestling.

Si presume che anche nelle nuove generazioni di wrestler, il conseguimento del titolo si attesterà attorno ai 33/34 anni.